In [ ]:
import numpy as np
import math
import cvxopt
import matplotlib.pyplot as plt

In [ ]:
# the first experiment about the offline QP
class Submodular_QP:
    noise=None
#we use the Submodular_QP.constraint to record the constraint matrix A, i.e., Ax\\\\le b\n",
    constraints=None
# we take QP as f(x)=1/2x^{T}Hx+h^{T}x\
    def __init__(self):
        self.H=None
        self.h=None
    def generate(self,n):
    # n is the dimension of x=(x_{1},...,x_{n})
        self.H=np.random.rand(n,n)
        self.H=0.5*(self.H+self.H.T)
        self.H=-self.H
        self.h=-self.H.dot(np.ones((n,1)))
    def stochastic_gradient_oracle(self,x):
        # return a stochastic gradient g=\\\\nable f(x)+\\\\deta*U[-1,1] for E(g)=\\\\nable f(x)=Hx+h\\n\",\n",
        noise=Submodular_QP.noise*np.random.randn(self.H.shape[0],1)
        return self.H.dot(x)+self.h+noise
    def stochastic_boosting_gradient_oracle(self,x):
        # we first generate a z~Z where Pr(Z\\\\le z)=(exp(z-1)-exp(-1))/(1-exp(-1))\\n\",\n",
        z=math.log((1-math.exp(-1))*np.random.rand()+math.exp(-1))+1
        return (1-math.exp(-1))*self.stochastic_gradient_oracle(z*x)
    def exact_value_oracle(self,x):
        # return f(x)
        return float(0.5*x.T.dot(self.H.dot(x))+self.h.T.dot(x))
    def set_constraints(m,n):
        Submodular_QP.constraints=np.random.rand(m,n)
    def set_noise(delta):
        Submodular_QP.noise=delta
    def set_delay(self,num):
        self.delay=np.random.randint(1,num+1)
        
        
#first method: traditional gradient ascent
def projected_gradient_ascent(H,T=500,batch=1):
    np.random.seed(T)
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    outcome=[0]*T
    point=np.zeros((n,1))
    #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c\\n\",\n",
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
    P=cvxopt.matrix(2*np.eye(n))
    for i in range(T):
        gradient=sum(H.stochastic_gradient_oracle(point) for j in range(batch))/batch
        point+=gradient/math.sqrt(i+1)
        q=cvxopt.matrix(-2*point)
        sol=cvxopt.solvers.qp(P,q,G,g)
        point=np.array(sol['x'])
        point=point.reshape(n,1)
        outcome[i]=H.exact_value_oracle(point)
    return outcome
#second method: boosting gradient ascent
def boosting_gradient_ascent(H,T=500,batch=1):
    np.random.seed(T)
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    outcome=[0]*T
    point=np.zeros((n,1))
    #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c\
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m+n,1)),np.zeros((n,1))),axis=0))
    P=cvxopt.matrix(2*np.eye(n))
    for i in range(T):
        gradient=sum(H.stochastic_boosting_gradient_oracle(point) for j in range(batch))/batch
        point+=gradient/math.sqrt(i+1)
        q=cvxopt.matrix(-2*point)
        sol=cvxopt.solvers.qp(P,q,G,g)
        point=np.array(sol['x'])
        point=point.reshape(n,1)
        outcome[i]=H.exact_value_oracle(point)
    return outcome
#the third method:conditional gradient ascent
def continuous_greedy_method(H,T=500):
    np.random.seed(T)
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    outcome=[0]*T
    point=np.zeros((n,1))
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
    for i in range(T):
        gradient=H.stochastic_gradient_oracle(point)
        sol=cvxopt.solvers.lp(cvxopt.matrix(-gradient),G,g)
        point=point+np.array(sol['x'])/T
        outcome[i]=H.exact_value_oracle(point)
    return outcome
#the fouth method: SCG
def stochastic_greedy_method(H,T=500):
    np.random.seed(T)
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    outcome=[0]*T
    point=np.zeros((n,1))
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
    gradient=H.stochastic_gradient_oracle(point)
    for i in range(T):
        rho=2/((i+4)**(2/3))
        gradient=(1-rho)*gradient+rho*H.stochastic_gradient_oracle(point)
        sol=cvxopt.solvers.lp(cvxopt.matrix(-gradient),G,g)
        point=point+np.array(sol['x'])/T
        outcome[i]=H.exact_value_oracle(point)
    return outcome
#the final method:SCG++
def stochastic_greedy_pp(H,T=500):
        np.random.seed(T)
        m=Submodular_QP.constraints.shape[0]
        n=Submodular_QP.constraints.shape[1]
        outcome=[0]*T
        point=np.zeros((n,1))
        G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
        g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
        g_t=sum(H.stochastic_gradient_oracle(point) for j in range(100**2))/(100**2)
        for i in range(T):
            gradient=cvxopt.matrix(-g_t)
            sol=cvxopt.solvers.lp(gradient,G,g)
            point1=np.array(sol['x'])/T
            point=point+point1
            outcome[i]=H.exact_value_oracle(point)
            g_t=g_t+H.H.dot(point1)
            #print(H.H.dot(point1))       
        return outcome

In [ ]:
#The second experiment: the online submodular QP
#before going into the detail of the algorithms,
#we want to realize a priority queue to record the information of delays.
class node:
    def __init__(self,delay=None,gradient=None):
        # we use the self.delay to record the delay term of self.gradient\n",
        self.delay=delay
        self.gradient=gradient
class Priority:
    def __init__(self):
        self.data=[]
    def __len__(self):
        return len(self.data)
    def insert(self,delay=None,gradient=None):
        self.data.append(node(delay,gradient))
        if len(self)==1:
            pass
        else:
            child=len(self)
            father=child//2
            while self.data[father-1].delay>self.data[child-1].delay:
                self.data[father-1],self.data[child-1]=self.data[child-1],self.data[father-1]
                child=father
                father=child//2
                if child==1:
                    break
    def show(self):
        return self.data[0]
    def minus_one(self):
        for i in range(len(self.data)):
            self.data[i].delay-=1
    def pop(self):
        data=self.data[0]
        if len(self)==1:
            self.data=[]
        else:
            self.data[0]=self.data.pop()
            father=1
            child=father*2
            setting=True
            while setting:
                if child<=len(self.data):
                    mini_child=child
                    if child+1<=len(self.data):
                        if self.data[mini_child-1].delay>self.data[child].delay:
                            mini_child=child+1
                    if self.data[father-1].delay>self.data[mini_child-1].delay:
                        self.data[father-1],self.data[mini_child-1]=self.data[mini_child-1],self.data[father-1]
                        father=mini_child
                        child=father*2
                    else:
                        setting=False
                else:
                    setting=False
        return data
    
#the first method: online gradient ascent
def online_gradient_ascent(problems,batch=50):
            m=Submodular_QP.constraints.shape[0]
            n=Submodular_QP.constraints.shape[1]
            T=len(problems)
            outcome=[0]*T
            point=np.zeros((n,1))
            #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c
            G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
            g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
            P=cvxopt.matrix(2*np.eye(n))
            # we use the priority queue to record the delay information
            Q=Priority()
            for i in range(T):
                outcome[i]=problems[i].exact_value_oracle(point)
                gradient=sum(problems[i].stochastic_gradient_oracle(point) for j in range(batch))/batch
                Q.insert(problems[i].delay,gradient)
                Q.minus_one()
                while len(Q)>0 and Q.show().delay==0:
                    point+=Q.pop().gradient/math.sqrt(T)
                q=cvxopt.matrix(-2*point)
                sol=cvxopt.solvers.qp(P,q,G,g)
                point=np.array(sol['x'])
                point=point.reshape(n,1)
            return outcome
        
#the second method:boosting gradient ascent
def boosting_online_gradient_ascent(problems,batch=50):
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    T=len(problems)
    outcome=[0]*T
    point=np.zeros((n,1))
    #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
    P=cvxopt.matrix(2*np.eye(n))
    # we use the priority queue to record the delay information\n",
    Q=Priority()
    for i in range(T):
        outcome[i]=problems[i].exact_value_oracle(point)
        gradient=sum(problems[i].stochastic_boosting_gradient_oracle(point) for j in range(batch))/batch
        Q.insert(problems[i].delay,gradient)
        Q.minus_one()
        while len(Q)>0 and Q.show().delay==0:
            point+=Q.pop().gradient/math.sqrt(T)
        q=cvxopt.matrix(-2*point)
        sol=cvxopt.solvers.qp(P,q,G,g)
        point=np.array(sol['x'])
        point=point.reshape(n,1)
    return outcome
        
#The third method: meta-FW
def meta_FW(problems,K=50):
            m=Submodular_QP.constraints.shape[0]
            n=Submodular_QP.constraints.shape[1]
            T=len(problems)
            outcome=[0]*T
            K_value=[np.zeros((n,1))]*K
            #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c\n",
            G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
            g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
            P=cvxopt.matrix(2*np.eye(n))
            # we use the priority queue to record the delay information\n",
            Q=[Priority() for i in range(K)]
            for i in range(T):
                point=sum(K_value)/K
                outcome[i]=problems[i].exact_value_oracle(point)
                for j in range(K):
                    gradient=problems[i].stochastic_gradient_oracle(sum(K_value[:(j+1)])/K)
                    Q[j].insert(problems[i].delay,gradient)
                    Q[j].minus_one()
                    while len(Q[j])>0 and Q[j].show().delay==0:
                        K_value[j]+=Q[j].pop().gradient/math.sqrt(T)
                    q=cvxopt.matrix(-2*K_value[j])
                    sol=cvxopt.solvers.qp(P,q,G,g)
                    K_value[j]=np.array(sol['x'])
                    K_value[j]=K_value[j].reshape(n,1)
            return outcome
#the final method: meta-FW-VR
def meta_FW_VR(problems,K=50):
            m=Submodular_QP.constraints.shape[0]
            n=Submodular_QP.constraints.shape[1]
            T=len(problems)
            outcome=[0]*T
            K_value=[np.zeros((n,1))]*K
            #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c\n",
            G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
            g=cvxopt.matrix(np.concatenate((np.ones((m,1)),np.ones((n,1)),np.zeros((n,1))),axis=0))
            P=cvxopt.matrix(2*np.eye(n))
            # we use the priority queue to record the delay information\n",
            Q=[Priority() for i in range(K)]
            for i in range(T):
                point=sum(K_value)/K
                outcome[i]=problems[i].exact_value_oracle(point)
                d_t=np.zeros((n,1))
                for j in range(K):
                    rho=2.2/(j+4)**(2/3)
                    gradient=problems[i].stochastic_gradient_oracle(sum(K_value[:(j+1)])/K)
                    Q[j].insert(problems[i].delay,gradient)
                    Q[j].minus_one()
                    gradient_mid=np.zeros((n,1))
                    while len(Q[j])>0 and Q[j].show().delay==0:
                        gradient_mid+=Q[j].pop().gradient/math.sqrt(T)
                    d_t=(1-rho)*(d_t)+rho*gradient_mid
                    q=cvxopt.matrix(-2*(K_value[j]+d_t))
                    sol=cvxopt.solvers.qp(P,q,G,g)
                    K_value[j]=np.array(sol['x'])
                    K_value[j]=K_value[j].reshape(n,1)
            return outcome
def optimal_T(problems):
    T=len(problems)
    m=Submodular_QP.constraints.shape[0]
    n=Submodular_QP.constraints.shape[1]
    outcome=[0]*T
    #setting the projection QP as 1/2x^{T}Px+q^{T}x s.t Gx \\\\le g, Cx=c\
    G=cvxopt.matrix(np.concatenate((Submodular_QP.constraints,np.eye(n),-np.eye(n)),axis=0))
    g=cvxopt.matrix(np.concatenate((np.ones((m+n,1)),np.zeros((n,1))),axis=0))
    P=cvxopt.matrix(2*np.eye(n))
    for i in range(T):
        point=np.zeros((n,1))
        for j in range(100): 
            gradient=sum(sum(problems[k].stochastic_boosting_gradient_oracle(point) for k in range(i+1)) for z in range(50))/50
            point+=gradient/math.sqrt(j+1)
            q=cvxopt.matrix(-2*point)
            sol=cvxopt.solvers.qp(P,q,G,g)
            point=np.array(sol['x'])
            point=point.reshape(n,1)
        outcome[i]=sum(problems[k].exact_value_oracle(point) for k in range(i+1))
    return outcome   